<a href="https://colab.research.google.com/github/anish-maddipatla/Quantitative-Portfolio/blob/main/Momentum_and_Value_Screener.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Momentum Screener

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
from tqdm import tqdm
from datetime import datetime, timedelta
from google.colab import files
import time

uploaded = files.upload()
stocks = pd.read_csv(next(iter(uploaded)))

tickers = stocks['Ticker'].tolist()
final_data_M = []
for ticker in tickers:
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        history = stock.history(period="1y")

        # Calculate momentum metrics
        if not history.empty:

            current_price = history['Close'].iloc[-1]

            # 12M-1M Return
            if len(history) >= 253:  # ~1 year of trading days
                price_12m_ago = history['Close'].iloc[-253]
                price_1m_ago = history['Close'].iloc[-21] if len(history) >= 21 else None
                return_12m_1m = (price_1m_ago - price_12m_ago) / price_12m_ago if price_1m_ago else None
            else:
                return_12m_1m = None

            # 6M Return
            if len(history) >= 126:  # ~6 months of trading days
                price_6m_ago = history['Close'].iloc[-126]
                return_6m = (current_price - price_6m_ago) / price_6m_ago
            else:
                return_6m = None

            # 3M Return
            if len(history) >= 63:  # ~3 months of trading days
                price_3m_ago = history['Close'].iloc[-63]
                return_3m = (current_price - price_3m_ago) / price_3m_ago
            else:
                return_3m = None

            # Volatility-adjusted 3M Return (Sharpe-like Ratio)
            if len(history) >= 63:
                daily_returns = history['Close'].pct_change().dropna()
                vol_3m = daily_returns.std() * np.sqrt(252)  # annualized volatility
                vol_adj_return_3m = return_3m / vol_3m if vol_3m != 0 else None
            else:
                vol_adj_return_3m = None

            # Price / 200DMA
            if len(history) >= 200:
                dma_200 = history['Close'].rolling(window=200).mean().iloc[-1]
                price_to_200dma = current_price / dma_200 if dma_200 != 0 else None
            else:
                price_to_200dma = None
        else:
            return_12m_1m = return_6m = return_3m = vol_adj_return_3m = price_to_200dma = None


        final_data_M.append({
            'Ticker': ticker,
            'Company Name': info.get('shortName'),
            'Sector': info.get('sector'),
            'Current Price': info.get('currentPrice'),
            'Market Cap': info.get('marketCap'),
            'P/E Ratio': info.get('trailingPE'),
            '52 Week High': info.get('fiftyTwoWeekHigh'),
            '52 Week Low': info.get('fiftyTwoWeekLow'),
            '12M-1M Return': return_12m_1m,
            '6M Return': return_6m,
            '3M Return': return_3m,
            'Sharpe-Like Ratio': vol_adj_return_3m,
            'Price / 200DMA': price_to_200dma
        })
        time.sleep(0.1)
    except Exception as err:
        print(f"Error fetching {ticker}: {err}")


df_momentum = pd.DataFrame(final_data_M)
df_momentum = df_momentum.dropna(subset=['Current Price'])

df_momentum['6M Return Score'] = 100*df_momentum['6M Return'].rank(pct=True)
df_momentum['3M Return Score'] = 100*df_momentum['3M Return'].rank(pct=True)
df_momentum['Sharpe-Like Ratio Score'] = 100*df_momentum['Sharpe-Like Ratio'].rank(pct=True)
df_momentum['Price / 200DMA Score'] = 100*df_momentum['Price / 200DMA'].rank(pct=True)

weights_momentum = {
    '6M Return Score': 0.2,
    '3M Return Score': 0.3,
    'Sharpe-Like Ratio Score': 0.3,
    'Price / 200DMA Score': 0.2
}

df_momentum['Momentum Score'] = (
    df_momentum['6M Return Score'] * weights_momentum['6M Return Score'] + df_momentum['3M Return Score'] * weights_momentum['3M Return Score'] +
    df_momentum['Sharpe-Like Ratio Score'] * weights_momentum['Sharpe-Like Ratio Score'] + df_momentum['Price / 200DMA Score'] * weights_momentum['Price / 200DMA Score']
)

df_momentum=df_momentum.sort_values(by='Momentum Score', ascending=False)
df_momentum.to_excel('Nifty50_momentum_scores.xlsx', index=False)
files.download('Nifty50_momentum_scores.xlsx')

Saving nifty_50_stocks.csv to nifty_50_stocks (1).csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Value Screener

In [ ]:
final_data_V=[]
valuation_data = []
profitability_data = []
cashflow_data = []
solvency_data = []

# --- Piotroski F-Score Function ---
def calculate_piotroski_score(financials, balance_sheet, cashflow, ticker):
    try:
        if financials.shape[1] < 2 or balance_sheet.shape[1] < 2 or cashflow.shape[1] < 2:
            return None

        def get_val(df, label):
            return df.loc[label].iloc[0] if label in df.index else None

        def get_prior(df, label):
            return df.loc[label].iloc[1] if label in df.index else None

        net_income = get_val(financials, 'Net Income')
        f1 = 1 if net_income and net_income > 0 else 0

        total_assets = get_val(balance_sheet, 'Total Assets')
        roa = net_income / total_assets if net_income and total_assets else None
        f2 = 1 if roa and roa > 0 else 0

        cfo = get_val(cashflow, 'Total Cash From Operating Activities')
        f3 = 1 if cfo and cfo > 0 else 0
        f4 = 1 if cfo and net_income and cfo > net_income else 0

        ltd_curr = get_val(balance_sheet, 'Long Term Debt')
        ltd_prev = get_prior(balance_sheet, 'Long Term Debt')
        f5 = 1 if ltd_curr is not None and ltd_prev is not None and ltd_curr < ltd_prev else 0

        curr_assets = get_val(balance_sheet, 'Current Assets')
        curr_liab = get_val(balance_sheet, 'Current Liabilities')
        prev_assets = get_prior(balance_sheet, 'Current Assets')
        prev_liab = get_prior(balance_sheet, 'Current Liabilities')
        curr_ratio = curr_assets / curr_liab if curr_assets and curr_liab else None
        prev_ratio = prev_assets / prev_liab if prev_assets and prev_liab else None
        f6 = 1 if curr_ratio and prev_ratio and curr_ratio > prev_ratio else 0

        shares_curr = get_val(balance_sheet, 'Ordinary Shares Number')
        shares_prev = get_prior(balance_sheet, 'Ordinary Shares Number')
        f7 = 1 if shares_curr and shares_prev and shares_curr <= shares_prev else 0

        gp_curr = get_val(financials, 'Gross Profit')
        sales_curr = get_val(financials, 'Total Revenue')
        gm_curr = gp_curr / sales_curr if gp_curr and sales_curr else None
        gp_prev = get_prior(financials, 'Gross Profit')
        sales_prev = get_prior(financials, 'Total Revenue')
        gm_prev = gp_prev / sales_prev if gp_prev and sales_prev else None
        f8 = 1 if gm_curr and gm_prev and gm_curr > gm_prev else 0

        ato_curr = sales_curr / total_assets if sales_curr and total_assets else None
        total_assets_prev = get_prior(balance_sheet, 'Total Assets')
        ato_prev = sales_prev / total_assets_prev if sales_prev and total_assets_prev else None
        f9 = 1 if ato_curr and ato_prev and ato_curr > ato_prev else 0

        return sum([f1, f2, f3, f4, f5, f6, f7, f8, f9])

    except Exception as e:
        print(f"Error calculating Piotroski F-Score for {ticker}: {e}")
        return None
for ticker in tickers:
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        financials = stock.financials
        balance_sheet = stock.balance_sheet
        cashflow = stock.cashflow

        # Metrics
        pe = info.get('trailingPE')
        pb = info.get('priceToBook')
        ev_ebitda = info.get('enterpriseToEbitda')
        earnings_yield = 1 / pe if pe else None
        roa = info.get('returnOnAssets')
        gross_margin = info.get('grossMargins')
        f_score = calculate_piotroski_score(financials, balance_sheet, cashflow, ticker)

        # Add to final data list
        final_data_V.append({
            'Ticker': ticker,
            'P/E': pe,
            'P/B': pb,
            'EV/EBITDA': ev_ebitda,
            'Earnings Yield': earnings_yield,
            'ROA': roa,
            'Gross Margin': gross_margin,
            'Piotroski F': f_score
        })

        time.sleep(1)

    except Exception as err:
        print(f"Error fetching {ticker}: {err}")
        final_data_V.append({
            'Ticker': ticker,
            'P/E': None,
            'P/B': None,
            'EV/EBITDA': None,
            'Earnings Yield': None,
            'ROA': None,
            'Gross Margin': None,
            'Piotroski F': None
        })
def normalize_scores(df, columns, higher_better=True):
    for col in columns:
        if higher_better:
            df[f'{col}_Score'] = df[col].rank(pct=True) * 100
        else:
            df[f'{col}_Score'] = (1 - df[col].rank(pct=True)) * 100
    return df


df_scored = pd.DataFrame(final_data_V)


df_scored = normalize_scores(df_scored, ['P/E', 'P/B', 'EV/EBITDA'], higher_better=False)
df_scored = normalize_scores(df_scored, ['Earnings Yield', 'ROA', 'Gross Margin', 'Piotroski F'], higher_better=True)


weights = {
    'P/E_Score': 0.10,
    'P/B_Score': 0.05,
    'EV/EBITDA_Score': 0.25,
    'Earnings Yield_Score': 0.10,
    'ROA_Score': 0.15,
    'Gross Margin_Score': 0.10,
    'Piotroski F_Score': 0.25
}


df_scored['Composite_Score'] = sum(df_scored[col] * weight for col, weight in weights.items())


df_scored['Percentile_Rank'] = df_scored['Composite_Score'].rank(pct=True) * 100


output_cols = ['Ticker', 'Composite_Score', 'Percentile_Rank'] + list(weights.keys())


df_export = df_scored[output_cols].sort_values('Composite_Score', ascending=False)
df_export.to_excel('Value_Strategy_Ranked.xlsx', index=False)


from google.colab import files
files.download('Value_Strategy_Ranked.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
def get_portfolio_size():
    while True:
        try:
            portfolio_size = float(input("Enter the total portfolio size in INR: Rs."))
            if portfolio_size <= 0:
                print("Portfolio size must be a positive number.")
                continue
            return portfolio_size
        except ValueError:
            print("Please enter a valid number for the portfolio size.")

# Use existing DataFrames instead of uploading
momentum_scores = df_momentum
value_scores = df_export

# Combine scores
combined_scores = pd.merge(
    momentum_scores[['Ticker', 'Momentum Score']],
    value_scores[['Ticker', 'Composite_Score']],
    on='Ticker'
)

combined_scores['Combined_Score'] = (
    0.5 * combined_scores['Momentum Score'] +
    0.5 * combined_scores['Composite_Score']
)

# Rank by combined score
combined_scores = combined_scores.sort_values('Combined_Score', ascending=False)

# Load stock data (only need to upload once)
uploaded = files.upload()
stocks = pd.read_csv(next(iter(uploaded)))  # Read the first uploaded file
tickers = stocks['Ticker'].tolist()
final_data = []

for ticker in tickers:
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        final_data.append({
            'Ticker': ticker,
            'Company Name': info.get('shortName'),
            'Sector': info.get('sector'),
            'Current Price': info.get('currentPrice'),
            'Market Cap': info.get('marketCap'),
        })
        time.sleep(0.1)
    except Exception as err:
        print(f"Error fetching {ticker}: {err}")

df = pd.DataFrame(final_data)
df = df.dropna(subset=['Current Price'])

# Portfolio allocation logic
portfolio_size = get_portfolio_size()
num_companies = len(df)
position_size = portfolio_size / num_companies

df['Position Size'] = position_size
df['Shares to Buy'] = (df['Position Size'] / df['Current Price']).astype(int)

# Calculate leftover cash
total_allocated = (df['Shares to Buy'] * df['Current Price']).sum()
leftover_cash = portfolio_size - total_allocated

# Distribute leftover to top 10 stocks by combined score
top_stocks = combined_scores.head(10)['Ticker']
additional_per_stock = leftover_cash / 10

for ticker in top_stocks:
    if ticker in df['Ticker'].values:
        additional_shares = (additional_per_stock / df.loc[df['Ticker'] == ticker, 'Current Price']).astype(int)
        df.loc[df['Ticker'] == ticker, 'Shares to Buy'] += additional_shares

# Final calculations
df['Final Allocation'] = df['Shares to Buy'] * df['Current Price']
total_invested = df['Final Allocation'].sum()
remaining_cash = portfolio_size - total_invested

# Add combined score to output
df = pd.merge(df, combined_scores, on='Ticker', how='left')

# Save results
output_cols = [
    'Ticker', 'Company Name', 'Sector', 'Current Price',
    'Shares to Buy', 'Final Allocation', 'Combined_Score',
    'Momentum Score', 'Composite_Score'
]
df[output_cols].to_excel("Nifty50_optimized_portfolio.xlsx", index=False)
files.download("Nifty50_optimized_portfolio.xlsx")

print("\nOptimized portfolio created successfully!")
print(f"Total portfolio size: Rs.{portfolio_size:,.2f}")
print(f"Total invested: Rs.{total_invested:,.2f}")
print(f"Remaining cash: Rs.{remaining_cash:,.2f}")
print(f"Top 10 stocks received additional allocation based on momentum-value score")

Saving nifty_50_stocks.csv to nifty_50_stocks (3).csv
Enter the total portfolio size in INR: Rs.100000


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Optimized portfolio created successfully!
Total portfolio size: Rs.100,000.00
Total invested: Rs.84,947.81
Remaining cash: Rs.15,052.19
Top 10 stocks received additional allocation based on momentum-value score
